In [1]:
import urllib.request as ur
import re 

f = ur.urlopen('https://resultados.as.com/resultados/futbol/primera/jornada')
s = f.read().decode()


In [2]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [3]:
from pprint import pprint     # import para hacer prints boninitos 

#Obtener url de cada jornada 
urlJornadas = re.findall('<li>.*?cont-jornada" href="(.*?)".*?</li>',s,re.DOTALL)

pprint(urlJornadas)

['/resultados/futbol/primera/2018_2019/jornada/regular_a_1/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_2/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_3/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_4/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_5/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_6/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_7/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_8/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_9/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_10/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_11/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_12/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_13/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_14/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_15/',
 '/resultados/futbol/primera/2018_2019/jornada/regular_a_16/',
 

In [4]:
import datetime

now = datetime.datetime.now()
months = ['Ago','Sep','Oct','Nov','Dic','Ene','Feb','Mar','Abr','May','Jun','Jul'] #Inicia en agosto como la temporada 
day = now.day
month = (now.month + 4) % 12



#f = ur.urlopen("https://resultados.as.com/resultados/futbol/primera/2018_2019/jornada/regular_a_1/")
#jornadai = f.read().decode()




    

# Para pasados 
def pasados():
    # Obtener Equipo1 GolesE1 GolesE2 Equipo2
    marcadorPasado = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?(\d\d* - \d\d*).*?</a>.*?class="nombre-equipo".*?>(.*?)</span>',jornadai,re.DOTALL)
    print(marcadorPasado)

    
# Para actuales 
# Obtener Equipo1 GolesE1 GolesE2 Equipo2 para los que lo tengan 
def actuales():
    listaResultados = jornadai.split('<li class="list-resultado">')
    listaResultados.pop(0)

    jugadosActual = []
    futurosActual = []

    for x in range(10):
        marcadorActual = []
        #print(len(marcadorActual))
        marcadorActual = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="resultado">.*?((..).-.(..)).*?</div>.*?class="nombre-equipo".*?>(.*?)</span>',listaResultados[x],re.DOTALL)
        print(marcadorActual)
        if marcadorActual[0][1] == '   -   ': 
            juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',listaResultados[x],re.DOTALL)
            futurosActual.append(juegosFuturos[0])
        else:
            jugadosActual.append(marcadorActual[0])

    print("actual jugado")
    pprint(jugadosActual)


    print("futuros en jornada actual")
    pprint(futurosActual)


# para los que no lo tengan, obtener Equipo1 Equipo2 FechaDeJuego

# Para futuros
# obtener Equipo1 Equipo2 FechaDeJuego
def futuros():
    juegosFuturos = re.findall('class="nombre-equipo".*?>(.*?)</span>.*?class="nombre-equipo".*?>(.*?)</span>.*?<span class="fecha">(.*?)</span>',jornadai,re.DOTALL)
    pprint(juegosFuturos)

In [ ]:

def printFuture(jornadaFecha): 
    if jornadaFecha[0][2] == '':
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
            
    else: 
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizará del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2]+ " al "+ jornadaFecha[0][3] +" de " + jornadaFecha[0][4] )
        else: 
            print("La " + jornadaFecha[0][0] + " se realizará el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )

def printPast(jornadaFecha):
    if jornadaFecha[0][2] == '':  
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
    else:
        if jornadaFecha[0][3] != '':
            print("La " + jornadaFecha[0][0] + " se realizó del " + jornadaFecha[0][1] + " de " + jornadaFecha[0][2] + " al "+ jornadaFecha[0][3] + " de " + jornadaFecha[0][4] )
        else: 
             print("La " + jornadaFecha[0][0] + " se realizó el " + jornadaFecha[0][1] + " de " + jornadaFecha[0][4] )
        

def printCurrent(jornadaFecha):
    print("La " + jornadaFecha[0][0] + " se está realizando")
            
#Abrir la jornada correspondiente para cada url 
for x in urlJornadas:
    f = ur.urlopen("https://resultados.as.com"+x)
    jornadai = f.read().decode()
    #print(s + "\n")
    #Obtener número de jornada y fecha 
    #jornadaFecha = re.findall('<h2 class="tit-modulo">.*?(Jornada \d*).*?>(\d\d) *-* *(\d\d)* (.*?)<.*?</h2>',jornadai,re.DOTALL)
    jornadaFecha = re.findall('<h2 class="tit-modulo">.*?(Jornada \d*).*?>(\d\d) (.*?) *- *(\d\d)* (.*?)<.*?</h2>',jornadai,re.DOTALL)
    print(jornadaFecha)
    

    if jornadaFecha[0][2] == '':  
        if months.index(jornadaFecha[0][4]) > month:
            printFuture(jornadaFecha)
            futuros()
        elif months.index(jornadaFecha[0][4]) < month:
            printPast(jornadaFecha)
            pasados()
        if months.index(jornadaFecha[0][4]) == month:
            if int(jornadaFecha[0][1]) > day:
                printFuture(jornadaFecha)
                futuros()
            if jornadaFecha[0][3] != '':
                if day <= int(jornadaFecha[0][3]) and  day >= int(jornadaFecha[0][1]):
                    printCurrent(jornadaFecha)
                    actuales()
                else: 
                    printPast(jornadaFecha)
                    pasados()
            elif int(jornadaFecha[0][1]) == day:
                printCurrent(jornadaFecha)
                actuales()
            else:
                printPast(jornadaFecha)
    else: 
        if months.index(jornadaFecha[0][2]) <= month and months.index(jornadaFecha[0][2]) >= month:
            print("current")
        elif months.index(jornadaFecha[0][2]) > month: 
            printPast(jornadaFecha)
            pasados()
        else:
            printFuture(jornadaFecha)
            futuros()
        
    print("\n\n\n")


[('Jornada 1', '17', '', '20', 'Ago')]
La Jornada 1 se realizó del 17 al 20 de Ago
[('Girona', '0 - 0', 'Valladolid'), ('Betis', '0 - 3', 'Levante'), ('Celta', '1 - 1', 'Espanyol'), ('Villarreal', '1 - 2', 'R. Sociedad'), ('Barcelona', '3 - 0', 'Alavés'), ('Eibar', '1 - 2', 'Huesca'), ('Rayo', '1 - 4', 'Sevilla'), ('Real Madrid', '2 - 0', 'Getafe'), ('Valencia', '1 - 1', 'Atlético'), ('Athletic', '2 - 1', 'Leganés')]




[('Jornada 2', '24', '', '27', 'Ago')]
La Jornada 2 se realizó del 24 al 27 de Ago
[('Getafe', '2 - 0', 'Eibar'), ('Leganés', '2 - 2', 'R. Sociedad'), ('Alavés', '0 - 0', 'Betis'), ('Atlético', '1 - 0', 'Rayo'), ('Valladolid', '0 - 1', 'Barcelona'), ('Espanyol', '2 - 0', 'Valencia'), ('Sevilla', '0 - 0', 'Villarreal'), ('Girona', '1 - 4', 'Real Madrid'), ('Levante', '1 - 2', 'Celta'), ('Athletic', '2 - 2', 'Huesca')]




[('Jornada 3', '31', 'Ago', '24', 'Oct')]
La Jornada 3 se realizará del 31 de Ago al 24 de Oct
[('Getafe', 'Valladolid', 'V-31/08 20:00'),
 ('Villarre

[('Jornada 20', '18', '', '21', 'Ene')]
La Jornada 20 se realizó del 18 al 21 de Ene
[('Getafe', '4 - 0', 'Alavés'), ('Real Madrid', '2 - 0', 'Sevilla'), ('Huesca', '0 - 3', 'Atlético'), ('Celta', '1 - 2', 'Valencia'), ('Betis', '3 - 2', 'Girona'), ('Villarreal', '1 - 1', 'Athletic'), ('Rayo', '2 - 2', 'R. Sociedad'), ('Levante', '2 - 0', 'Valladolid'), ('Barcelona', '3 - 1', 'Leganés'), ('Eibar', '3 - 0', 'Espanyol')]




[('Jornada 21', '26', '', '28', 'Ene')]
La Jornada 21 se realizó del 26 al 28 de Ene
[('Sevilla', '5 - 0', 'Levante'), ('Atlético', '2 - 0', 'Getafe'), ('Leganés', '2 - 2', 'Eibar'), ('Valencia', '3 - 0', 'Villarreal'), ('Valladolid', '2 - 1', 'Celta'), ('Girona', '0 - 2', 'Barcelona'), ('R. Sociedad', '0 - 0', 'Huesca'), ('Athletic', '1 - 0', 'Betis'), ('Espanyol', '2 - 4', 'Real Madrid'), ('Alavés', '0 - 1', 'Rayo')]




[('Jornada 22', '01', '', '04', 'Feb')]
La Jornada 22 se realizó del 01 al 04 de Feb
[('Huesca', '4 - 0', 'Valladolid'), ('Levante', '0 - 0', 'Get